In [1]:
import os, sys, glob
import pandas as pd
import json
import argparse
import time
import numpy
from tqdm import tqdm

# Import coffea specific features
from coffea import processor

# SUEP Repo Specific
from workflows import SUEP_coffea
from workflows import merger

In [2]:
out_dir = os.getcwd()
modules_era = []

modules_era.append(
    SUEP_coffea.SUEP_cluster(
        isMC=1,
        era=2018,
        do_syst=1,
        syst_var="",
        sample="SUEP_m400_darkPhoHad",
        weight_syst="",
        flag=False,
        output_location=out_dir,
    )
)

In [3]:
from dask.distributed import Client

client = Client("tls://christos-2epapageorgakis-40cern-2ech.dask.coffea.casa:8786")
client

<Client: 'tls://192.168.106.25:8786' processes=3 threads=6, memory=18.44 GiB>

In [4]:
import shutil
shutil.make_archive("workflows", "zip", base_dir="workflows")
client.upload_file("workflows.zip")

{'tls://christos-2epapageorgakis-40cern-2ech.dask-worker.coffea.casa:8788': {'status': 'OK'},
 'tls://red-c7126.unl.edu:41539': {'status': 'OK'},
 'tls://red-c7126.unl.edu:45245': {'status': 'OK'}}

In [ ]:
redirector = "root://xcache/"
path = (
    "/store/user/chpapage/SUEPnano/"
)

for instance in modules_era:
    executor = processor.DaskExecutor(
        client = client
    )
    #executor = processor.FuturesExecutor()
    run = processor.Runner(
        executor = executor,
        schema = processor.NanoAODSchema,
    )    
    output = run(
        {instance.sample: [redirector + path + "SUEP-m400-darkPhoHad.root"]},
        treename = "Events",
        processor_instance = instance,
    )

[####################                    ] | 50% Completed |  1min 26.3s

In [ ]:
merger.merge(1)